In [1]:
#!import AutomatedAnalysis.dib

  Determining projects to restore...
  All projects are up-to-date for restore.
  GC.Analysis.API -> Q:\performance\artifacts\bin\GC.Analysis.API\Release\net8.0\GC.Analysis.API.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:01.17


Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-public/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.1 Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

Loading extensions from `Q:\.tools\.nuget\packages\microsoft.data.analysis\0.19.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `Q:\.tools\.nuget\packages\xplot.plotly.interactive\4.0.7\lib\net7.0\XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-public/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.1 Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

In [6]:
string path = @".\Examples\Traces\All_Conserve";
DataManager dm = DataManager.CreateAspNetData(path);
string baselineConfigName = "28c";
string comparandConfigName = "28c_con1";

## Display Benchmark Regressions

In [7]:
Metric<BenchmarkData> avgHeapCountChanges = Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Average);
Metric<BenchmarkData> avgGCCount = Metrics.Promote(Metrics.I.TotalGCCount, Aggregation.Average);
public static Dictionary<Metric<BenchmarkData>, ExpectationDirection> HigherMemoryLowerThroughput = new()
{
    { Metrics.B.AverageMaxHeapSize, ExpectationDirection.Increase },
    { Metrics.B.AverageP50Latency, ExpectationDirection.Decrease },
    /*
    Additionally, a metric with no change is also available. The entire enum is:
    public enum ExpectationDirection
    {
        Unknown,
        Increase,
        Decrease,
        NoChange,
    }
    */
};

Dictionary< Metric<BenchmarkData>, double> thresholds = new()
{
    { Metrics.B.AverageMaxHeapSize, 5 },
    { Metrics.B.AverageP50Latency, 5 },
};

DisplayBenchmarkSummaryWithRegressions(dm, baselineConfigName, comparandConfigName, metrics: ML( Metrics.B.AverageMaxHeapSize, Metrics.B.AverageP50Latency ), benchmarkRegressionThreshold: 5, benchmarksToDisplay: 5, 
metricToleranceOverridePercentage: thresholds,
expectationDirections: HigherMemoryLowerThroughput);

## Out of 30 benchmarks, 12 have regressed.
   - __ConnectionClose__ has regressed by 24.77% for Average of Latency 50th
   - __JsonHttpSys__ has regressed by 17.54% for Average of Latency 50th
   - __Json__ has regressed by 12.09% for Average of Latency 50th
   - __Stage1Pgo__ has regressed by 5.31% for Average of Latency 50th
   - __ConnectionCloseHttps__ has regressed by 2.74% for Average of Latency 50th


### Per-benchmark behavior

|                  Benchmark Name | Average of Max heap size / 28c | Average of Max heap size / 28c_con1 | Average of Max heap size /  Comparison % | Average of Latency 50th / 28c | Average of Latency 50th / 28c_con1 | Average of Latency 50th /  Comparison % |
| ------------------------------: | -----------------------------: | ----------------------------------: | ---------------------------------------: | ----------------------------: | ---------------------------------: | --------------------------------------: |
|          ConnectionCloseHttpSys |                         21.115 |                              20.370 |                                   -3.530 |                         0.190 |                              0.190 |                                   0.000 |
|            ConnectionCloseHttps |                         17.201 |                              16.700 |                                   -2.911 |                         0.182 |                              0.188 |                                   2.740 |
|                 ConnectionClose |                          7.575 |                               7.178 |                                   -5.241 |                         1.100 |                              1.373 |                                  24.773 |
|                     JsonHttpSys |                         34.516 |                              48.246 |                                   39.780 |                         0.527 |                              0.620 |                                  17.536 |
|                    JsonPlatform |                          3.016 |                               3.030 |                                    0.475 |                         0.297 |                              0.300 |                                   0.840 |
|                            Json |                          4.110 |                               4.102 |                                   -0.186 |                         0.228 |                              0.255 |                                  12.088 |
|               PlaintextEndpoint |                          4.078 |                               4.090 |                                    0.292 |                         0.458 |                              0.460 |                                   0.546 |
|         PlaintextPlatformInline |                          7.521 |                               5.877 |                                  -21.858 |                         1.038 |                              1.045 |                                   0.723 |
|               PlaintextPlatform |                          5.158 |                               3.234 |                                  -37.314 |                         0.862 |                              0.853 |                                  -1.159 |
| PlaintextWithParametersNoFilter |                         36.313 |                              49.667 |                                   36.777 |                         0.648 |                              0.650 |                                   0.386 |
|                       Plaintext |                          4.105 |                               4.042 |                                   -1.530 |                         0.412 |                              0.412 |                                   0.000 |
|                       Stage1Pgo |                         30.603 |                              38.501 |                                   25.807 |                         0.283 |                              0.298 |                                   5.310 |

## Display Iteration Anomalies

In [17]:
// Update Average -> Safe Average.
Aggregation SafeAverage = new Aggregation((gc => {
    double safeAverage;
    
    long count = 0;
    double sum = 0;

    foreach (var item in gc)
    {
        if (item != 0 && !double.IsNaN(item))
        {
            sum += item;
            count++;
        }
    }

    safeAverage = count == 0 ? double.NaN : sum / count;
    return safeAverage;

}), "SafeAverage", "");

Metric<IterationData> avgTCPToConsider = Metrics.Promote(Metrics.G.MedianThroughputCostPercent, SafeAverage);

Dictionary<Metric<IterationData>, double> volatilityOverrides = new()
{
    { Metrics.I.Gen0MeanPauseMSec, 0.5 },
    { Metrics.I.Latency50thMS, 3 },
};

// todo: override the volatility threshold
DisplayIterationData(dm, baselineConfigName, comparandConfigName, metrics: ML( avgTCPToConsider, Metrics.I.Gen0MeanPauseMSec, Metrics.I.Latency50thMS ), benchmarkFilter: new Filter("Stage1Pgo"),
volatilityThreshold: 2, metricVolOverridePercentage: volatilityOverrides); 

## Anomalies in Iterations
   - __Stage1Pgo__ for SafeAverage of MedianThroughputCostPercent has an anomaly: 1.4 for 0th iteration with Volatility: 45.68% - 1.4, 2.04, 1.86, 1.83.
   - __Stage1Pgo__ for g0 Mean Pause has an anomaly: 1.49 for 3th iteration with Volatility: 20.74% - 1.24, 1.37, 1.26, 1.49.
   - __Stage1Pgo__ for Latency 50th has an anomaly: 0.27 for 3th iteration with Volatility: 18.52% - 0.32, 0.28, 0.32, 0.27.
   - __Stage1Pgo__ for Latency 50th has an anomaly: 0.3 for 0th iteration with Volatility: 11.11% - 0.3, 0.28, 0.27, 0.28.
   - __Stage1Pgo__ for g0 Mean Pause has an anomaly: 1.5 for 1th iteration with Volatility: 11.07% - 1.35, 1.5, 1.38, 1.38.


## Stage1Pgo

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 1.96 | 1.35 | 0.30 |
| 28c | 1 | 1.93 | 1.50 | 0.28 |
| 28c | 2 | 1.94 | 1.38 | 0.27 |
| 28c | 3 | 1.97 | 1.38 | 0.28 |
| 28c_con1 | 0 | 1.40 | 1.24 | 0.32 |
| 28c_con1 | 1 | 2.04 | 1.37 | 0.28 |
| 28c_con1 | 2 | 1.86 | 1.26 | 0.32 |
| 28c_con1 | 3 | 1.83 | 1.49 | 0.27 |
| 28c | Average | 1.95 | 1.40 | 0.28 |
| 28c_con1 | Average | 1.78 | 1.34 | 0.30 |
| Comparison | % Diff | -8.70 | -4.48 | 5.31 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 0 | SafeAverage of MedianThroughputCostPercent  | 45.68 | 1.4 | 1.40 | 2.04 | 1.86 | 1.83 | 
| 28c_con1 | 3 | g0 Mean Pause  | 20.74 | 1.49 | 1.24 | 1.37 | 1.26 | 1.49 | 
| 28c_con1 | 3 | Latency 50th  | 18.52 | 0.27 | 0.32 | 0.28 | 0.32 | 0.27 | 
| 28c | 0 | Latency 50th  | 11.11 | 0.3 | 0.30 | 0.28 | 0.27 | 0.28 | 
| 28c | 1 | g0 Mean Pause  | 11.07 | 1.5 | 1.35 | 1.50 | 1.38 | 1.38 | 
| 28c | 1 | SafeAverage of MedianThroughputCostPercent  | 2.07 | 1.93 | 1.96 | 1.93 | 1.94 | 1.97 | 


## Display GC Charts

In [15]:
// TODO: Ask mark about xArrangement work.

ChartGCData(dm, metrics: ML( Metrics.G.MedianThroughputCostPercent, Metrics.G.NumHeaps, Metrics.G.TotalSOHStableSize ), 
 //iterationFilter: new IntFilter(1), 
 benchmarkFilter: new Filter("PlaintextMvc", "Stage1Pgo"),
 xArrangement: XArrangements.Relative)

Too many units: %, #, bytes


<!DOCTYPE html>

<!DOCTYPE html>

index value 0 <!DOCTYPE html>